Referencias Importantes:

https://arxiv.org/pdf/1807.01868.pdf - Artículo de las Imagénes

https://pillow.readthedocs.io/en/stable/reference/Image.html - Librería PIL

https://www.geeksforgeeks.org/python-pil-image-save-method/ - Librería PIL 2

In [5]:
pip install hexbytes


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: /opt/anaconda3/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install datasets[vision]


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: /opt/anaconda3/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import re
import pandas as pd
import seaborn as sns
from hexbytes import HexBytes
import matplotlib.pyplot as plt
from datasets import load_dataset

In [2]:
# Due to a bug in the HuggingFace dataset, at the moment two file checksums do not correspond to what
# is in the dataset metadata, thus we have to load the data splits with the flag ignore_verification
# set to true
train_set = load_dataset("mwritescode/slither-audited-smart-contracts", 'big-multilabel', split='train', ignore_verifications=True)
test_set = load_dataset("mwritescode/slither-audited-smart-contracts", 'big-multilabel', split='test', ignore_verifications=True)
val_set = load_dataset("mwritescode/slither-audited-smart-contracts", 'big-multilabel', split='validation', ignore_verifications=True)
     


/opt/anaconda3/lib/python3.8/site-packages/datasets/load.py:1748: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(


Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/79641 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/15972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10861 [00:00<?, ? examples/s]

Dataset slither-audited-smart-contracts downloaded and prepared to /Users/Shanti/.cache/huggingface/datasets/mwritescode___slither-audited-smart-contracts/big-multilabel/1.1.0/4cf503b59ce9d3157914e47f6253de773b7ab828f46642685d4b470b88ca1f13. Subsequent calls will reuse this data.


Found cached dataset slither-audited-smart-contracts (/Users/Shanti/.cache/huggingface/datasets/mwritescode___slither-audited-smart-contracts/big-multilabel/1.1.0/4cf503b59ce9d3157914e47f6253de773b7ab828f46642685d4b470b88ca1f13)
Found cached dataset slither-audited-smart-contracts (/Users/Shanti/.cache/huggingface/datasets/mwritescode___slither-audited-smart-contracts/big-multilabel/1.1.0/4cf503b59ce9d3157914e47f6253de773b7ab828f46642685d4b470b88ca1f13)


Let's now define a couple of utility functions for the pre-precessing:

In [9]:
def remove_comments(string):
    pattern = r"(\".*?\"|\'.*?\')|(/\*.*?\*/|//[^\r\n]*$)"
    # first group captures quoted strings (double or single)
    # second group captures comments (//single-line or /* multi-line */)
    regex = re.compile(pattern, re.MULTILINE|re.DOTALL)
    def _replacer(match):
        # if the 2nd group is not None, then we have captured a real comment string.
        if match.group(2) is not None:
            return "" 
        else: # otherwise, we will return the 1st group
            return match.group(1) 
    return regex.sub(_replacer, string)

def get_lenghts(example):
    code = remove_comments(example['source_code'])
    example['sourcecode_len'] = len(code.split())
    example['bytecode_len'] = len(HexBytes(example['bytecode']))
    return example

Now we can use HuggingFace datasets map() function to apply the transformations we defined above to all our dataset splits. The same function also allows us to remove attributes that are useless for our Exploratory Data Analysis steps. Finally, we'll concatenate all the splits in a single pandas DataFrame, which is compatible with seaborn's plotting utilities. 

In [14]:
COLS_TO_REMOVE = ['source_code', 'bytecode']

LABELS = {0:'access-control', 1:'arithmetic', 2:'other', 3:'reentrancy', 4:'safe', 5:'unchecked-calls'}

datasets = []
for split in [train_set, test_set, val_set]:
    split_df = pd.DataFrame(split.map(get_lenghts, remove_columns=COLS_TO_REMOVE)).explode('slither')
    split_df['slither'] = split_df['slither'].map(LABELS)
    datasets.append(split_df)


Loading cached processed dataset at /Users/Shanti/.cache/huggingface/datasets/mwritescode___slither-audited-smart-contracts/big-multilabel/1.1.0/4cf503b59ce9d3157914e47f6253de773b7ab828f46642685d4b470b88ca1f13/cache-d5eb0b7f3e9688dd.arrow
Loading cached processed dataset at /Users/Shanti/.cache/huggingface/datasets/mwritescode___slither-audited-smart-contracts/big-multilabel/1.1.0/4cf503b59ce9d3157914e47f6253de773b7ab828f46642685d4b470b88ca1f13/cache-babed9aaf7578c73.arrow
Loading cached processed dataset at /Users/Shanti/.cache/huggingface/datasets/mwritescode___slither-audited-smart-contracts/big-multilabel/1.1.0/4cf503b59ce9d3157914e47f6253de773b7ab828f46642685d4b470b88ca1f13/cache-ecd71a5b391cc529.arrow


In [22]:
for split, split_name in zip(datasets, ['train', 'test', 'val']):
    ncatenated = pd.concat([split.assign(dataset=split_name)])

In [23]:
ncatenated.head(5)

,address,slither,sourcecode_len,bytecode_len,dataset
0,0x0518a3c95f777b585dde38814331077c325565b1,safe,724,5135,val
1,0x0a98ad316dfbcdaef91e382348a0c3639ec8f9d3,reentrancy,3808,10977,val
1,0x0a98ad316dfbcdaef91e382348a0c3639ec8f9d3,other,3808,10977,val
1,0x0a98ad316dfbcdaef91e382348a0c3639ec8f9d3,arithmetic,3808,10977,val
1,0x0a98ad316dfbcdaef91e382348a0c3639ec8f9d3,unchecked-calls,3808,10977,val


SMART CONTRACTS CON BYTECODE

In [24]:
def remove_comments(string):
    pattern = r"(\".*?\"|\'.*?\')|(/\*.*?\*/|//[^\r\n]*$)"
    # first group captures quoted strings (double or single)
    # second group captures comments (//single-line or /* multi-line */)
    regex = re.compile(pattern, re.MULTILINE|re.DOTALL)
    def _replacer(match):
        # if the 2nd group is not None, then we have captured a real comment string.
        if match.group(2) is not None:
            return "" 
        else: # otherwise, we will return the 1st group
            return match.group(1) 
    return regex.sub(_replacer, string)

def get_lenghts(example):
    code = remove_comments(example['source_code'])
    example['sourcecode_len'] = len(code.split())
    return example

In [25]:
COLS_TO_REMOVE = ['source_code']

LABELS = {0:'access-control', 1:'arithmetic', 2:'other', 3:'reentrancy', 4:'safe', 5:'unchecked-calls'}

datasets = []
for split in [train_set, test_set, val_set]:
    split_df = pd.DataFrame(split.map(get_lenghts, remove_columns=COLS_TO_REMOVE)).explode('slither')
    split_df['slither'] = split_df['slither'].map(LABELS)
    datasets.append(split_df)


Map:   0%|          | 0/79641 [00:00<?, ? examples/s]

Map:   0%|          | 0/15972 [00:00<?, ? examples/s]

Map:   0%|          | 0/10861 [00:00<?, ? examples/s]

In [26]:
for split, split_name in zip(datasets, ['train', 'test', 'val']):
    concatenated = pd.concat([split.assign(dataset=split_name)])

In [27]:
concatenated.head(5)

,address,bytecode,slither,sourcecode_len,dataset
0,0x0518a3c95f777b585dde38814331077c325565b1,0x6080604052600436106100c5576000357c0100000000...,safe,724,val
1,0x0a98ad316dfbcdaef91e382348a0c3639ec8f9d3,0x608060405234801561001057600080fd5b5060043610...,reentrancy,3808,val
1,0x0a98ad316dfbcdaef91e382348a0c3639ec8f9d3,0x608060405234801561001057600080fd5b5060043610...,other,3808,val
1,0x0a98ad316dfbcdaef91e382348a0c3639ec8f9d3,0x608060405234801561001057600080fd5b5060043610...,arithmetic,3808,val
1,0x0a98ad316dfbcdaef91e382348a0c3639ec8f9d3,0x608060405234801561001057600080fd5b5060043610...,unchecked-calls,3808,val


In [250]:
import math
import os
from PIL import Image

def closestDivisors(n):
    a = round(math.sqrt(n))
    while n%a > 0: a -= 1
    return a,n//a

In [252]:
a = 0
for i in concatenated['bytecode']:
    i = str(i)
    byte = bytes(i,'utf-8')
    if (len(byte)/3) > 1:
        width, height = closestDivisors(int(len(byte)/3))
        #print(a, width, height)
        img = Image.frombytes("RGB", (width, height), byte)
        name = 'byte_imgs/bytecode_img_'+str(a)+'.png'
        img.save(name)
        a += 1
    else:
        a += 1

In [262]:
import os
byte_test = str(concatenated['bytecode'][11])
byte_test = bytes(byte_test, 'utf-8')
#print(len(byte_test)/3)
#print(math.sqrt(len(byte_test)/3))
width, height = closestDivisors(int(len(byte_test)/3))
print(width, height)

65.66666666666667
5 13


In [226]:
from PIL import Image
import io
import base64
#from bitarray import byte_data
#byte_test = io.BytesIO(
    #base64.b64decode(
        #byte_test
    #)
#)
#print(byte_test)
#io.BytesIO(byte_test)

#Image.open(byte_test)
img = Image.frombytes("RGB", (width, height), byte_test)
    
img.save("byte_imgs/pitoanal.png")